In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../../../rings_varying_radius_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 16
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 16

MCMC_SIZE = 50
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-3

CUDA = torch.cuda.is_available()
PATH = 'ep-rad-init-eta'
DEVICE = torch.device('cuda:0')

noise_sigma = torch.ones(1) * 0.05
if CUDA:
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (noise_sigma, N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
oneshot_eta = Oneshot_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)

optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
# enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
# enc_eta.load_state_dict(torch.load("../weights/enc-mu-%s" % PATH))
# oneshot_eta.load_state_dict(torch.load("../weights/oneshot-mu-%s" % PATH))

In [ ]:
from ag_ep_rad import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\1000 (116s),  symKL_DB_eta: 378219.768,  symKL_DB_z: 183424.157,  gap: 561319.422,  loss: -95856.770,  ess: 2.348
epoch: 1\1000 (114s),  symKL_DB_eta: 130937.893,  symKL_DB_z: 139181.093,  gap: 270015.733,  loss: -47145.227,  ess: 2.728
epoch: 2\1000 (114s),  symKL_DB_eta: 47855.182,  symKL_DB_z: 94375.288,  gap: 142176.955,  loss: -33846.869,  ess: 3.392
epoch: 3\1000 (114s),  symKL_DB_eta: 31886.123,  symKL_DB_z: 45865.406,  gap: 77703.048,  loss: -28320.518,  ess: 3.891
epoch: 4\1000 (114s),  symKL_DB_eta: 26208.121,  symKL_DB_z: 29102.649,  gap: 55257.629,  loss: -26647.406,  ess: 4.132
epoch: 5\1000 (114s),  symKL_DB_eta: 24264.864,  symKL_DB_z: 19257.556,  gap: 43474.115,  loss: -25718.781,  ess: 4.326
epoch: 6\1000 (114s),  symKL_DB_eta: 22423.486,  symKL_DB_z: 13550.641,  gap: 35932.785,  loss: -25056.917,  ess: 4.484
epoch: 7\1000 (114s),  symKL_DB_eta: 21955.730,  symKL_DB_z: 10072.207,  gap: 31980.595,  loss: -25228.878,  ess: 4.594
epoch: 8\1000 (114s),  symKL_DB_e

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-ep-rad-init-eta")
torch.save(enc_eta.state_dict(), "../weights/enc-mu-ep-rad-init-eta")
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-mu-ep-rad-init-eta")

In [ ]:
from ag_ep_rad import *

BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
Model_Params_Test = (noise_sigma, N, K, D, 20)
obs, metric_step, reused = test(models, EUBO_init_eta, Data, Model_Params_Test, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')